## APPLICATION TEMPLATE ERA5 - DAILY MEAN TEMPERATURE

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
import xarray as xr

import c3s_eqc_toolbox_template

import warnings
warnings.filterwarnings('ignore')


### REQUEST DEFINITION

In [ ]:
request_2021 = {
    'product_type': 'reanalysis',
    'format': 'grib',
    'year': '2021',
    'month': [
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12'
    ],
    'day': [
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12',
        '13', '14', '15',
        '16', '17', '18',
        '19', '20', '21',
        '22', '23', '24',
        '25', '26', '27',
        '28', '29', '30',
        '31',
    ],
    'time': [
        '06', '18' 
    ],
    'variable': '2m_temperature',
}

request_2022 = {
    'product_type': 'reanalysis',
    'format': 'grib',
    'year': '2022',
    'month': [
        '01', '02', '03',
        '04', '05', '06',
        '07', '08'
    ],
    'day': [
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12',
        '13', '14', '15',
        '16', '17', '18',
        '19', '20', '21',
        '22', '23', '24',
        '25', '26', '27',
        '28', '29', '30',
        '31',
    ],
    'time': [
        '06', '18'
    ],
    'variable': '2m_temperature',
}

collection_id = 'reanalysis-era5-single-levels'
requests = [request_2021, request_2022]

### PROCESSING with Xarray: GLOBAL DAILY MEAN TEMPERATURE 2021-2022 

#### Define operation to be applied to each single chunk

In [ ]:
def spatial_daily_mean(ds: xr.Dataset) ->  xr.Dataset:
    ds = c3s_eqc_toolbox_template.statistics.spatial_mean(ds)
    return ds.resample(time="1D").mean("time")

#### Perform the processing


For each chunk:
    
- Download chunk
- apply the operation `f` to the chunk
- cache the transformed chunk

Merge all the transformed chunk

In [ ]:
daily_global_t2m = c3s_eqc_toolbox_template.download_and_transform(
    collection_id, requests, chunks={"year": 1,"month": 1}, f=spatial_daily_mean, open_with="xarray"
)

daily_global_t2m

#### Plot result 

In [ ]:
daily_global_t2m = daily_global_t2m.squeeze()
fig = px.line(
    x=daily_global_t2m["time"],
    y=daily_global_t2m["t2m"] - 273.15, 
)
fig.update_layout(
    xaxis_title="time", 
    yaxis_title="t2m", 
    title="GLOBAL DAILY MEAN TEMPERATURE 2021-2022"
)
fig.show()

In [ ]:
fig.write_image("ERA5_global_daily_mean_temperature_2021-2022.png")

### PROCESSING with pandas: DAILY MEAN TEMPERATURE ON ROME 2021-2022 

#### Define operation to be applied to each single chunk

In [ ]:
def daily_mean_on_rome(df: pd.DataFrame)-> pd.DataFrame:
    lat, lon = 42, 12.5
    df = df.reset_index()
    df = df[(df.latitude == 42) & (df.longitude == 12.5)]
    return df.groupby(df.valid_time.dt.floor("d")).mean()

#### Perform the processing

In [ ]:
daily_rome_t2m = c3s_eqc_toolbox_template.download_and_transform(
    collection_id, requests, chunks={"year": 1,"month": 1}, f=daily_mean_on_rome, open_with="pandas")


daily_rome_t2m 

#### Plot result 

In [ ]:
daily_global_t2m = daily_global_t2m.squeeze()
fig = px.line(daily_rome_t2m  - 273.15, y="t2m")

fig.update_layout(
    xaxis_title="time", 
    yaxis_title="t2m", 
    title="DAILY MEAN TEMPERATURE ON ROME 2021-2022"
)

fig.show()

In [ ]:
fig.write_image("ERA5_daily_mean_temperature_on_rome_2021-2022.png")